In [18]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from google.colab import files
from zipfile import ZipFile
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet169

In [23]:
file_name = 'face-mask-dataset.zip'

with ZipFile(file_name, 'r') as zip:
    zip.extractall()

In [24]:
batch_size = 40
img_height = 200
img_width = 200

In [25]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    'data',
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 7553 files belonging to 2 classes.
Using 6043 files for training.


In [26]:
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    'data',
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 7553 files belonging to 2 classes.
Using 1510 files for validation.


In [27]:
densenet_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

In [28]:
densenet_model.trainable = False

In [29]:
densenet_model.summary()

Model: "densenet169"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 206, 206, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 100, 100, 64) 9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 100, 100, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [30]:
model = tf.keras.models.Sequential([
    densenet_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')                                
])

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Functional)     (None, 6, 6, 1664)        12642880  
_________________________________________________________________
flatten_1 (Flatten)          (None, 59904)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               15335680  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 27,978,817
Trainable params: 15,335,937
Non-trainable params: 12,642,880
_________________________________________________________________


In [32]:
model.compile(optimizer=Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
model.fit(train_data, validation_data=test_data, epochs=5)

Epoch 1/5
152/152 [==============================] - 35s 182ms/step - loss: 2.2198 - accuracy: 0.6645 - val_loss: 0.4104 - val_accuracy: 0.8424
Epoch 2/5
152/152 [==============================] - 25s 163ms/step - loss: 0.5587 - accuracy: 0.7995 - val_loss: 0.3383 - val_accuracy: 0.8682
Epoch 3/5
152/152 [==============================] - 25s 159ms/step - loss: 0.3630 - accuracy: 0.8542 - val_loss: 0.2908 - val_accuracy: 0.8848
Epoch 4/5
152/152 [==============================] - 25s 162ms/step - loss: 0.2967 - accuracy: 0.8872 - val_loss: 0.2737 - val_accuracy: 0.8980
Epoch 5/5
152/152 [==============================] - 25s 160ms/step - loss: 0.2526 - accuracy: 0.9069 - val_loss: 0.2706 - val_accuracy: 0.8974


In [34]:
densenet_model.trainable = True
trainable = False
for layer in densenet_model.layers:
    if layer.name == 'conv1_block32':
        trainable = True
    layer.trainable = trainable

In [35]:
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-5), metrics=['accuracy'])

In [37]:
model.fit(train_data, validation_data=test_data, epochs=5)

Epoch 1/5
152/152 [==============================] - 26s 167ms/step - loss: 0.1452 - accuracy: 0.9429 - val_loss: 0.2219 - val_accuracy: 0.9245
Epoch 2/5
152/152 [==============================] - 25s 164ms/step - loss: 0.1200 - accuracy: 0.9535 - val_loss: 0.2414 - val_accuracy: 0.9225
Epoch 3/5
152/152 [==============================] - 26s 167ms/step - loss: 0.1123 - accuracy: 0.9553 - val_loss: 0.2127 - val_accuracy: 0.9318
Epoch 4/5
152/152 [==============================] - 25s 163ms/step - loss: 0.0886 - accuracy: 0.9654 - val_loss: 0.2031 - val_accuracy: 0.9404
Epoch 5/5
152/152 [==============================] - 26s 167ms/step - loss: 0.0834 - accuracy: 0.9707 - val_loss: 0.2139 - val_accuracy: 0.9318
